In [10]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta, UTC
from tqdm import tqdm
from dotenv import load_dotenv
from binance.client import Client

In [ ]:
# === Загрузка API ключей из .env ===
load_dotenv()
api_key = os.getenv("ENV_API_KEY")
api_secret = os.getenv("ENV_SECRET_KEY")

# === Инициализация клиента Binance ===
client = Client(api_key, api_secret)

# === Параметры загрузки ===
symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_5MINUTE # можно менять (1MINUTE, 1HOUR и т.д.)
start_str = "1 Sep 2020"
start_time = datetime.strptime(start_str, "%d %b %Y").replace(tzinfo=UTC)
end_time = datetime.now(UTC)

if interval == Client.KLINE_INTERVAL_1MINUTE:
    delta = timedelta(minutes=1000)
elif interval == Client.KLINE_INTERVAL_5MINUTE:
    delta = timedelta(minutes=5000)
elif interval == Client.KLINE_INTERVAL_15MINUTE:
    delta = timedelta(minutes=15000)
elif interval == Client.KLINE_INTERVAL_1HOUR:
    delta = timedelta(hours=1000)
elif interval == Client.KLINE_INTERVAL_4HOUR:
    delta = timedelta(hours=4000)
elif interval == Client.KLINE_INTERVAL_1DAY:
    delta = timedelta(days=1000)
else:
    raise ValueError("⚠️ Добавь поддержку для этого интервала")

#delta_unit = 60 if "MINUTE" in interval else 3600  # для tqdm

# === Прогрессбар ===
total_steps = int((end_time - start_time) / delta)
print("⏳ Загружаем данные с Binance по 1000 свечей за раз...")
pbar = tqdm(total=total_steps)

all_klines = []
current = start_time

# === Загрузка свечей ===
while current < end_time:
    next_time = current + delta
    klines = client.get_historical_klines(
        symbol,
        interval,
        current.strftime("%d %b %Y %H:%M:%S"),
        next_time.strftime("%d %b %Y %H:%M:%S")
    )

    if not klines:
        break

    all_klines.extend(klines)
    current = next_time
    pbar.update(1)
    time.sleep(0.2)  # не долбим API

pbar.close()
print(f"✅ Загружено свечей: {len(all_klines)}")

# === Преобразуем в DataFrame ===
df = pd.DataFrame(all_klines, columns=[
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base_vol", "taker_buy_quote_vol", "ignore"
])

df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
df["close_time"] = pd.to_datetime(df["close_time"], unit="ms")

numeric_cols = ["open", "high", "low", "close", "volume", "quote_asset_volume"]
df[numeric_cols] = df[numeric_cols].astype(float)
df.rename(columns={"quote_asset_volume": "turnover"}, inplace=True)

# === Оставим ключевые колонки ===
df = df[["open_time", "open", "high", "low", "close", "volume", "turnover"]]

# === Сохранение в файл ===
filename = f"{symbol}_{interval}_from_{start_str.replace(' ', '_')}.csv"
df.to_csv(filename, index=False)
print(f"📁 Данные сохранены в: {filename}")


In [ ]:
df.shape